In [1]:
from functions import *


# Torch cannot work properly in jupyter notebook
# os.chdir("test_dir")


[nltk_data] Downloading package stopwords to C:\Users\Zoe
[nltk_data]     Lua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
final_df = pd.read_pickle("Data/final_df.pkl")

In [3]:
import os
os.getcwd()


'c:\\Users\\Zoe Lua\\DSA4266_Grp2'

## Data preprocessing

In [4]:
def remove(text):

    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

    # tokens = ' '.join([''.join(c for c in word if c not in string.punctuation) for word in tokens if word])

    return tokens

final_df['cleaned'] = final_df.combined.apply(remove)


## To try:

- GloVe + LSTM + Ray Tune (Hyperpara Tune)
<!-- - GloVe + LSTM  -->
- subword tokenization

## Datasets:
- Combined subjects with 

In [5]:
import torch

print(torch.__version__)

2.2.1+cpu


## Train test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(final_df.drop('class', axis = 1), final_df['class'], test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)

(402, 3)
(198, 3)


## GloVe

In [7]:
glove_file = "GloVe/glove.6B.300d.txt"
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

C:\Users\Zoe Lua\AppData\Local\Temp\ipykernel_6324\3503967038.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


In [8]:
# sample = final_df.cleaned[0]
# sample

In [9]:
final_df

,class,contains_img,combined,cleaned
0,1,0,re num luxury watches buy your own rolex for...,"[num, luxury, watches, buy, rolex, money, role..."
1,1,0,take a moment to explore this academic qualifi...,"[take, moment, explore, academic, qualificatio..."
2,0,0,greetings greetings all this is to verify your...,"[greetings, greetings, verify, subscription, p..."
3,1,0,loans num num num term try chauncey may co...,"[loans, num, num, num, term, try, chauncey, ma..."
4,0,0,who wants to start its quiet too quiet well ho...,"[wants, start, quiet, quiet, well, straw, poll..."
...,...,...,...,...
595,1,0,shenzhen jianshen industrial co ltd dear compa...,"[shenzhen, jianshen, industrial, co, ltd, dear..."
596,0,0,re the thing that annoys me the most about sev...,"[thing, annoys, seven, swords, although, joke,..."
597,1,0,company business invoicing on behalf of the co...,"[company, business, invoicing, behalf, company..."
598,1,0,shenzhen jianshen industrial co ltd dear compa...,"[shenzhen, jianshen, industrial, co, ltd, dear..."


In [10]:

def embed_fn(x):
    embed = []
    for token in x:
        if token in glove_vectors:
            embed.append(glove_vectors[token])
        else:
            embed += [0]
            print(token + " NOT FOUND!")

    return embed

final_df['vector'] = final_df.cleaned.apply(embed_fn)
final_df

sohne NOT FOUND!
jaegerlecoultre NOT FOUND!
exapmle NOT FOUND!
nonacc NOT FOUND!
redited NOT FOUND!
versities NOT FOUND!
helors NOT FOUND!
ifications NOT FOUND!
rendleman NOT FOUND!
haertel NOT FOUND!
dharap NOT FOUND!
colarelli NOT FOUND!
siebenmann NOT FOUND!
rdist NOT FOUND!
youve NOT FOUND!
sparcs NOT FOUND!
embrittle NOT FOUND!
kkpt NOT FOUND!
kkpt NOT FOUND!
nevadamarket NOT FOUND!
wireapr NOT FOUND!
otckkptpk NOT FOUND!
wwwwellenhancementcom NOT FOUND!
alkalinesurfactantpolymer NOT FOUND!
vtaltum NOT FOUND!
casset NOT FOUND!
casset NOT FOUND!
russianbased NOT FOUND!
studentsadults NOT FOUND!
residented NOT FOUND!
usdmonth NOT FOUND!
malefemale NOT FOUND!
nwkplxh NOT FOUND!
nwkplxh NOT FOUND!
embrittle NOT FOUND!
contenttype NOT FOUND!
textplain NOT FOUND!
charsetiso NOT FOUND!
formatflowed NOT FOUND!
contenttransferencoding NOT FOUND!
contenttype NOT FOUND!
texthtml NOT FOUND!
charsetiso NOT FOUND!
contenttransferencoding NOT FOUND!
ninebark NOT FOUND!
assimilable NOT FOUND!
dow

,class,contains_img,combined,cleaned,vector
0,1,0,re num luxury watches buy your own rolex for...,"[num, luxury, watches, buy, rolex, money, role...","[[-0.2471, 0.14174, 0.38344, 0.17708, -0.22988..."
1,1,0,take a moment to explore this academic qualifi...,"[take, moment, explore, academic, qualificatio...","[[-0.015879, 0.11807, -0.12769, -0.16302, -0.0..."
2,0,0,greetings greetings all this is to verify your...,"[greetings, greetings, verify, subscription, p...","[[-1.0159, -0.62914, -0.050402, -0.34164, -1.1..."
3,1,0,loans num num num term try chauncey may co...,"[loans, num, num, num, term, try, chauncey, ma...","[[0.22987, -0.17886, -0.10652, -0.49421, 0.199..."
4,0,0,who wants to start its quiet too quiet well ho...,"[wants, start, quiet, quiet, well, straw, poll...","[[-0.18316, -0.058127, -0.29759, -0.15329, 0.1..."
...,...,...,...,...,...
595,1,0,shenzhen jianshen industrial co ltd dear compa...,"[shenzhen, jianshen, industrial, co, ltd, dear...","[[0.034999, 0.33392, 0.46038, -0.53346, 0.0319..."
596,0,0,re the thing that annoys me the most about sev...,"[thing, annoys, seven, swords, although, joke,...","[[0.26464, -0.10502, -0.014347, 0.033417, 0.14..."
597,1,0,company business invoicing on behalf of the co...,"[company, business, invoicing, behalf, company...","[[0.021777, 0.14174, -0.035402, -0.22559, -0.2..."
598,1,0,shenzhen jianshen industrial co ltd dear compa...,"[shenzhen, jianshen, industrial, co, ltd, dear...","[[0.034999, 0.33392, 0.46038, -0.53346, 0.0319..."


## RNN

pad > train rnn

In [89]:
from transformers import Tokenizer

46